# L2HMC for $U(1)$ Gauge Model 

## Imports

In [1]:
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.special import i0, i1

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from l2hmc_eager import dynamics_eager as _l2hmc
from l2hmc_eager import gauge_dynamics_eager as l2hmc
from l2hmc_eager.neural_nets import *
from utils.distributions import GMM, gen_ring
from utils.jacobian import _map, jacobian
from utils.data_utils import (
    calc_avg_vals_errors, block_resampling, jackknife_err
)

%autoreload 2

Using TensorFlow backend.


In [2]:
from lattice.ising_lattice import IsingLattice
from lattice.gauge_lattice import GaugeLattice, u1_plaq_exact

In [3]:
#from l2hmc_eager.neural_nets import _custom_conv
from l2hmc_eager.neural_nets import _custom_dense

In [4]:
tf.enable_eager_execution()
tfe = tf.contrib.eager

## Use GaugeModelEager for training L2HMC on $U(1)$ gauge lattice

In [5]:
from l2hmc_eager.u1_model_eager import GaugeModelEager

In [29]:
params = {
    'time_size': 8,
    'space_size': 8,
    'link_type': 'U1',
    'dim': 2,
    'beta': 3.5,
    'num_samples': 2,
    'n_steps': 5,
    'eps': 0.2,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate_init': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.96,
    'train_iters': 500,
    'record_loss_every': 50,
    'data_steps': 10,
    'save_steps': 50,
    'print_steps': 1,
    'logging_steps': 5,
    'clip_value': 100,
    'rand': False,
    'metric': 'l2',
    'conv_net': True,
    'hmc': False
}


In [ ]:
eps_arr = np.linspace(0.05, 0.5, 10)
n_steps_arr = [5, 10, 15]

for eps in eps_arr:
    for n_steps in n_steps_arr:
        params['eps'] = eps
        params['n_steps'] = n_steps
        
        tf.reset_default_graph()
        model = GaugeModelEager(params=params,
                                log_dir=None,
                                restore=False,
                                use_defun=True)
        _ = model.calc_observables(model.samples, _print=True, _write=True)
        model.train(1)
        
        model.train(1000)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_eager/run_25/
Creating lattice...
done.
Creating dynamics...
done.
total initialization time: 0.27184200286865234



step: 0   loss:   0     time/step:   0     accept:   0     eps:  0.05   avg_action:   0     avg_top_charge:   0     avg_plaq:   1    

Training complete.
Saved checkpoint to: /Users/saforem2/ANL/l2hmc/gauge_logs_eager/run_25/ckpt-1
avg_plaquettes: [1. 1.]



step: 1   loss: -45621  time/step: 1348.4  accept: 0.82544  eps: 0.0499  avg_action: 12.961  avg_top_charge: -0.087596  avg_plaq: 0.94214 



step: 2   loss: -41278  time/step: 2.4379  accept:   1     eps: 0.049829  avg_action: 19.98   avg_top_charge: 0.041591  avg_plaq: 0.9108 



step: 3   loss: -36983  time/step: 1.8555  accept: 0.95698  eps: 0.049746  avg_action: 20.845  avg_top_charge: 0.025737  avg_plaq: 0.90694 



step: 4   loss: -32072  time/step: 1.5768  accept: 0.63952  eps: 0.049658  avg_action: 23.238  avg_top_charge: 0.026996  avg_plaq



step: 55  loss: -24799  time/step: 2.7873  accept:   1     eps: 0.049377  avg_action:  37.5   avg_top_charge: 0.0093997  avg_plaq: 0.83259 



step: 56  loss: -26702  time/step: 2.2716  accept: 0.93659  eps: 0.04944  avg_action: 38.859  avg_top_charge: -0.072798  avg_plaq: 0.82652 



step: 57  loss: -27946  time/step: 2.3698  accept:   1     eps: 0.049497  avg_action: 39.012  avg_top_charge: -0.085668  avg_plaq: 0.82584 



step: 58  loss: -27009  time/step: 2.1341  accept: 0.59795  eps: 0.049548  avg_action: 42.443  avg_top_charge: -0.037973  avg_plaq: 0.81052 



step: 59  loss: -28742  time/step: 2.2522  accept: 0.96855  eps: 0.04959  avg_action: 38.116  avg_top_charge: -0.040633  avg_plaq: 0.82984 



step: 60  loss: -21221  time/step: 1.9516  accept: 0.55495  eps: 0.049648  avg_action: 39.741  avg_top_charge: -0.014263  avg_plaq: 0.82258 



step: 61  loss: -21093  time/step: 2.0082  accept: 0.34494  eps: 0.049712  avg_action: 43.787  avg_top_charge: -0.073336  avg_plaq: 0.8045



step: 112 loss: -21431  time/step: 1.907   accept: 0.13491  eps: 0.051506  avg_action: 44.66   avg_top_charge: 0.048513  avg_plaq: 0.80062 



step: 113 loss: -26049  time/step: 1.8538  accept: 0.22254  eps: 0.051544  avg_action: 44.66   avg_top_charge: 0.048513  avg_plaq: 0.80062 



step: 114 loss: -29792  time/step: 2.5081  accept: 0.35109  eps: 0.051577  avg_action: 49.237  avg_top_charge: 0.12611  avg_plaq: 0.78019 



step: 115 loss: -25644  time/step: 2.0838  accept: 0.53431  eps: 0.051606  avg_action: 51.475  avg_top_charge: 0.09572  avg_plaq: 0.7702 



step: 116 loss: -25134  time/step: 2.7506  accept: 0.72298  eps: 0.051631  avg_action: 48.777  avg_top_charge: 0.14037  avg_plaq: 0.78225 



step: 117 loss: -26071  time/step: 2.2527  accept: 0.064135  eps: 0.051653  avg_action: 48.777  avg_top_charge: 0.14037  avg_plaq: 0.78225 



step: 118 loss: -23564  time/step: 2.349   accept: 0.56898  eps: 0.05167  avg_action: 49.654  avg_top_charge: 0.1787  avg_plaq: 0.77833 



step



step: 169 loss: -17038  time/step: 4.858   accept: 0.58188  eps: 0.053114  avg_action: 42.455  avg_top_charge: -0.042239  avg_plaq: 0.81047 



step: 170 loss: -25346  time/step: 3.1066  accept: 0.1982  eps: 0.053121  avg_action: 42.455  avg_top_charge: -0.042239  avg_plaq: 0.81047 



step: 171 loss: -23169  time/step: 2.9157  accept: 0.65707  eps: 0.053141  avg_action: 50.122  avg_top_charge: -0.0763  avg_plaq: 0.77624 



step: 172 loss: -26715  time/step: 2.7342  accept: 0.48457  eps: 0.053156  avg_action: 49.688  avg_top_charge: 0.033043  avg_plaq: 0.77818 



step: 173 loss: -26398  time/step: 2.3817  accept: 0.87126  eps: 0.053169  avg_action: 49.084  avg_top_charge: 0.059122  avg_plaq: 0.78088 



step: 174 loss: -25887  time/step: 2.6316  accept: 0.19105  eps: 0.053179  avg_action: 49.084  avg_top_charge: 0.059122  avg_plaq: 0.78088 



step: 175 loss: -28289  time/step: 2.2934  accept: 0.589   eps: 0.053187  avg_action: 44.077  avg_top_charge: 0.034326  avg_plaq: 0.80323 





step: 226 loss: -26417  time/step: 2.5324  accept:   1     eps: 0.053374  avg_action: 45.082  avg_top_charge: -0.08622  avg_plaq: 0.79874 



step: 227 loss: -26407  time/step: 2.2516  accept: 0.65315  eps: 0.053365  avg_action: 43.667  avg_top_charge: -0.072011  avg_plaq: 0.80506 



step: 228 loss: -29480  time/step: 2.838   accept: 0.76847  eps: 0.053357  avg_action: 40.046  avg_top_charge: -0.055962  avg_plaq: 0.82122 



step: 229 loss: -27028  time/step: 2.3696  accept: 0.54938  eps: 0.053347  avg_action: 40.046  avg_top_charge: -0.055962  avg_plaq: 0.82122 



step: 230 loss: -29646  time/step: 2.4321  accept:   1     eps: 0.053339  avg_action: 34.298  avg_top_charge: -0.048995  avg_plaq: 0.84689 



step: 231 loss: -28991  time/step: 2.5317  accept:   1     eps: 0.053332  avg_action: 38.907  avg_top_charge: -0.063865  avg_plaq: 0.82631 



step: 232 loss: -27509  time/step: 2.1689  accept:   1     eps: 0.053325  avg_action: 39.437  avg_top_charge: -0.023695  avg_plaq: 0.82394

...



step: 269 loss: -27320  time/step: 10.228  accept:   1     eps: 0.039689  avg_action: 39.783  avg_top_charge: 0.037706  avg_plaq: 0.8224 



step: 270 loss: -27134  time/step: 12.436  accept:   1     eps: 0.039675  avg_action: 48.159  avg_top_charge: 0.011421  avg_plaq: 0.78501 



step: 271 loss: -26724  time/step: 6.0402  accept:   1     eps: 0.039662  avg_action: 54.203  avg_top_charge: 0.07675  avg_plaq: 0.75802 



step: 272 loss: -23787  time/step: 6.8068  accept: 0.55185  eps: 0.039649  avg_action: 40.696  avg_top_charge: 0.17445  avg_plaq: 0.81832 



step: 273 loss: -27076  time/step: 6.303   accept: 0.52665  eps: 0.039636  avg_action: 39.915  avg_top_charge: -0.054914  avg_plaq: 0.82181 



step: 274 loss: -24519  time/step: 6.5879  accept:   1     eps: 0.039624  avg_action: 39.225  avg_top_charge: 0.063757  avg_plaq: 0.82489 



step: 275 loss: -29513  time/step: 5.7323  accept: 0.94488  eps: 0.039609  avg_action: 41.154  avg_top_charge: -0.074338  avg_plaq: 0.81628 



st



step: 326 loss: -27187  time/step: 7.0501  accept: 0.54627  eps: 0.038603  avg_action: 39.943  avg_top_charge: -0.19474  avg_plaq: 0.82169 



step: 327 loss: -28854  time/step: 65.765  accept: 0.5164  eps: 0.038588  avg_action: 30.891  avg_top_charge: 0.017599  avg_plaq: 0.86209 



step: 328 loss: -27053  time/step: 25.072  accept:   1     eps: 0.038574  avg_action: 34.335  avg_top_charge: 0.09389  avg_plaq: 0.84672 



step: 329 loss: -30751  time/step: 6.684   accept: 0.88803  eps: 0.038558  avg_action: 39.117  avg_top_charge: 0.021117  avg_plaq: 0.82537 



step: 330 loss: -27988  time/step: 6.2537  accept:   1     eps: 0.038543  avg_action: 41.16   avg_top_charge: 0.024545  avg_plaq: 0.81625 



step: 331 loss: -27793  time/step: 6.6741  accept: 0.54747  eps: 0.038527  avg_action: 45.888  avg_top_charge: 0.010343  avg_plaq: 0.79514 



step: 332 loss: -24274  time/step: 6.4137  accept: 0.093544  eps: 0.038512  avg_action: 45.888  avg_top_charge: 0.010343  avg_plaq: 0.79514 







step: 383 loss: -28189  time/step: 5.448   accept: 0.99831  eps: 0.037622  avg_action: 44.92   avg_top_charge: -0.08674  avg_plaq: 0.79946 



step: 384 loss: -28282  time/step: 5.8795  accept: 0.58386  eps: 0.037609  avg_action: 44.981  avg_top_charge: 0.039875  avg_plaq: 0.79919 



step: 385 loss: -25776  time/step: 9.6484  accept:   1     eps: 0.037597  avg_action: 42.941  avg_top_charge: -0.1183  avg_plaq: 0.8083 



step: 386 loss: -26757  time/step: 10.48   accept: 0.86652  eps: 0.037579  avg_action: 39.503  avg_top_charge: -0.062483  avg_plaq: 0.82365 



step: 387 loss: -24322  time/step: 6.4766  accept: 0.6971  eps: 0.037555  avg_action: 43.422  avg_top_charge: -0.041169  avg_plaq: 0.80615 



step: 388 loss: -28356  time/step: 5.8655  accept:   1     eps: 0.037533  avg_action: 40.267  avg_top_charge: -0.023414  avg_plaq: 0.82024 



step: 389 loss: -30305  time/step: 5.7953  accept:   1     eps: 0.037512  avg_action: 45.586  avg_top_charge: 0.039198  avg_plaq: 0.79649 







step: 440 loss: -26421  time/step: 4.2959  accept: 0.86352  eps: 0.036588  avg_action: 40.245  avg_top_charge: -0.086576  avg_plaq: 0.82033 



step: 441 loss: -31451  time/step: 4.6624  accept:   1     eps: 0.036577  avg_action: 37.036  avg_top_charge: 0.066282  avg_plaq: 0.83466 



step: 442 loss: -27141  time/step: 4.4184  accept: 0.77493  eps: 0.036565  avg_action: 44.58   avg_top_charge: 0.035551  avg_plaq: 0.80098 



step: 443 loss: -27726  time/step: 4.5654  accept:   1     eps: 0.036553  avg_action: 44.294  avg_top_charge: -0.045268  avg_plaq: 0.80226 



step: 444 loss: -24458  time/step: 4.8043  accept:   1     eps: 0.036542  avg_action: 34.239  avg_top_charge: 0.12059  avg_plaq: 0.84715 



step: 445 loss: -24388  time/step: 5.2773  accept: 0.90561  eps: 0.036527  avg_action: 36.15   avg_top_charge: -0.035219  avg_plaq: 0.83862 



step: 446 loss: -30043  time/step: 5.0468  accept:   1     eps: 0.036512  avg_action: 43.704  avg_top_charge: 0.013797  avg_plaq: 0.80489 






step: 497 loss: -27113  time/step: 5.2545  accept:   1     eps: 0.035842  avg_action: 38.225  avg_top_charge: 0.058121  avg_plaq: 0.82935 



step: 498 loss: -25064  time/step: 5.6688  accept:   1     eps: 0.035825  avg_action: 34.222  avg_top_charge: 0.061495  avg_plaq: 0.84722 



step: 499 loss: -27712  time/step: 5.5775  accept:   1     eps: 0.035809  avg_action: 32.223  avg_top_charge: 0.038399  avg_plaq: 0.85615 



step: 500 loss: -27171  time/step: 6.1886  accept: 0.83679  eps: 0.035791  avg_action: 36.116  avg_top_charge: -0.089816  avg_plaq: 0.83877 

Saved checkpoint to: /Users/saforem2/ANL/l2hmc/gauge_logs_eager/run_27/ckpt-11
avg_plaquettes: [0.8432127  0.83431953]



step: 501 loss: -23608  time/step: 7.1415  accept:   1     eps: 0.035775  avg_action: 39.735  avg_top_charge: 0.015901  avg_plaq: 0.82261 



step: 502 loss: -26210  time/step: 7.8606  accept:   1     eps: 0.035759  avg_action: 35.976  avg_top_charge: 0.022402  avg_plaq: 0.83939 



step: 503 loss: -24731  

In [64]:
tf.reset_default_graph()

In [56]:
model  = GaugeModelEager(params=params,
                         log_dir=None,
                         restore=False,
                         use_defun=True)

Creating directory for new run: /Users/saforem2/ANL/l2hmc/gauge_logs_eager/run_19/
Creating lattice...
done.
Creating dynamics...
done.
total initialization time: 0.11740612983703613



In [50]:
observables = model.calc_observables(model.samples, _print=True, _write=True)
total_actions, avg_plaquettes, top_charges = observables



step: 0   loss:   0     time/step:   0     accept:   0     eps:  0.2    avg_action:   0     avg_top_charge:   0     avg_plaq:   1    



In [65]:
model.train(1000, keep_samples=False)



step: 200 loss: -29937  time/step: 8.8363  accept: 0.26475  eps: 0.19407  avg_action: 11.288  avg_top_charge: -0.032814  avg_plaq: 0.94961 

Saved checkpoint to: /Users/saforem2/ANL/l2hmc/gauge_logs_eager/run_19/ckpt-5
avg_plaquettes: [0.8992153 1.       ]



step: 201 loss: -33893  time/step: 2.1382  accept: 0.62555  eps: 0.19397  avg_action: 16.75   avg_top_charge: 0.029704  avg_plaq: 0.92523 



step: 202 loss: -30722  time/step: 3.3576  accept: 0.51364  eps: 0.19387  avg_action: 23.216  avg_top_charge: 0.024194  avg_plaq: 0.89636 



step: 203 loss: -30091  time/step: 1.7112  accept: 0.51506  eps: 0.19379  avg_action: 18.484  avg_top_charge: 0.015068  avg_plaq: 0.91748 



step: 204 loss: -39707  time/step: 1.1589  accept:   1     eps: 0.1937  avg_action: 34.297  avg_top_charge: 0.020611  avg_plaq: 0.84689 



step: 205 loss: -35350  time/step: 1.4864  accept:   1     eps: 0.19363  avg_action: 43.049  avg_top_charge: -0.049421  avg_plaq: 0.80782 



step: 206 loss: -35053  time/s



step: 256 loss: -30357  time/step: 0.62815  accept:   1     eps: 0.19364  avg_action: 43.087  avg_top_charge: 0.12284  avg_plaq: 0.80765 



step: 257 loss: -26092  time/step: 0.61269  accept: 0.87656  eps: 0.19373  avg_action: 46.354  avg_top_charge: 0.13519  avg_plaq: 0.79306 



step: 258 loss: -27613  time/step: 0.53732  accept: 0.54106  eps: 0.1938  avg_action: 43.14   avg_top_charge: 0.0028052  avg_plaq: 0.80741 



step: 259 loss: -30243  time/step: 0.6149  accept: 0.96269  eps: 0.19386  avg_action: 42.231  avg_top_charge: 0.019756  avg_plaq: 0.81147 



step: 260 loss: -28785  time/step: 0.67677  accept:   1     eps: 0.19391  avg_action: 40.756  avg_top_charge: 0.0067871  avg_plaq: 0.81805 



step: 261 loss: -27036  time/step: 0.51987  accept: 0.58445  eps: 0.19396  avg_action: 43.885  avg_top_charge: 0.042657  avg_plaq: 0.80409 



step: 262 loss: -26239  time/step: 0.57089  accept: 0.64173  eps: 0.19401  avg_action: 42.866  avg_top_charge: -0.0075019  avg_plaq: 0.80863 






step: 313 loss: -26565  time/step: 0.82184  accept: 0.3475  eps: 0.19395  avg_action: 36.696  avg_top_charge: 0.037559  avg_plaq: 0.83618 



step: 314 loss: -27387  time/step: 0.62909  accept: 0.65314  eps: 0.19393  avg_action: 38.311  avg_top_charge: -0.023043  avg_plaq: 0.82897 



step: 315 loss: -27562  time/step: 0.54227  accept:   1     eps: 0.19392  avg_action: 35.524  avg_top_charge: 0.025897  avg_plaq: 0.84141 



step: 316 loss: -28073  time/step: 0.65135  accept:   1     eps: 0.19391  avg_action: 42.562  avg_top_charge: 0.017712  avg_plaq: 0.80999 



step: 317 loss: -26438  time/step: 0.80421  accept: 0.68247  eps: 0.1939  avg_action: 48.138  avg_top_charge: 0.05235  avg_plaq: 0.7851 



step: 318 loss: -26051  time/step: 0.75014  accept: 0.87598  eps: 0.19388  avg_action: 48.51   avg_top_charge: -0.048452  avg_plaq: 0.78344 



step: 319 loss: -28306  time/step: 0.76307  accept: 0.87023  eps: 0.19387  avg_action: 40.88   avg_top_charge: -0.027225  avg_plaq: 0.8175 



s



step: 370 loss: -25838  time/step: 0.71378  accept: 0.50807  eps: 0.19325  avg_action: 38.453  avg_top_charge: -0.091677  avg_plaq: 0.82834 



step: 371 loss: -26673  time/step: 0.67585  accept: 0.80375  eps: 0.19324  avg_action: 38.116  avg_top_charge: 0.01844  avg_plaq: 0.82984 



step: 372 loss: -29738  time/step: 23.051  accept: 0.90346  eps: 0.19323  avg_action: 39.132  avg_top_charge: -0.013102  avg_plaq: 0.8253 



step: 373 loss: -25192  time/step: 1.5994  accept: 0.80678  eps: 0.19321  avg_action: 38.94   avg_top_charge: 0.071085  avg_plaq: 0.82616 



step: 374 loss: -23602  time/step: 0.5089  accept: 0.6235  eps: 0.19319  avg_action: 37.652  avg_top_charge: 0.0051279  avg_plaq: 0.83191 



step: 375 loss: -25397  time/step: 0.82269  accept: 0.31149  eps: 0.19318  avg_action: 37.652  avg_top_charge: 0.0051279  avg_plaq: 0.83191 



step: 376 loss: -26056  time/step: 1.5438  accept: 0.42149  eps: 0.19317  avg_action: 37.181  avg_top_charge: 0.038346  avg_plaq: 0.83401 







step: 427 loss: -28114  time/step: 0.51687  accept: 0.72392  eps: 0.19237  avg_action:  39.7   avg_top_charge: -0.054178  avg_plaq: 0.82277 



step: 428 loss: -25102  time/step: 0.74199  accept: 0.62473  eps: 0.19236  avg_action: 35.901  avg_top_charge: 0.02647  avg_plaq: 0.83973 



step: 429 loss: -27844  time/step: 0.69309  accept: 0.61759  eps: 0.19235  avg_action: 33.979  avg_top_charge: -0.065968  avg_plaq: 0.84831 



step: 430 loss: -25583  time/step: 0.81852  accept: 0.42282  eps: 0.19233  avg_action: 36.524  avg_top_charge: -0.072251  avg_plaq: 0.83695 



step: 431 loss: -25496  time/step: 0.60731  accept: 0.64714  eps: 0.19231  avg_action: 38.209  avg_top_charge: -0.061871  avg_plaq: 0.82942 



step: 432 loss: -25716  time/step: 0.52299  accept: 0.62036  eps: 0.19229  avg_action: 36.711  avg_top_charge: 0.028197  avg_plaq: 0.83611 



step: 433 loss: -27296  time/step: 0.75003  accept: 0.89416  eps: 0.19227  avg_action: 28.68   avg_top_charge: 0.011172  avg_plaq: 0.8719

...



step: 768 loss: -28347  time/step: 0.60741  accept: 0.5798  eps: 0.1881  avg_action: 39.93   avg_top_charge: -0.030929  avg_plaq: 0.82174 



step: 769 loss: -31400  time/step: 0.57262  accept:   1     eps: 0.18809  avg_action: 42.48   avg_top_charge: 0.048709  avg_plaq: 0.81036 



step: 770 loss: -27817  time/step: 0.49307  accept: 0.81831  eps: 0.18808  avg_action: 41.019  avg_top_charge: -0.033947  avg_plaq: 0.81688 



step: 771 loss: -26399  time/step: 0.57813  accept:   1     eps: 0.18807  avg_action: 40.829  avg_top_charge: -0.0075839  avg_plaq: 0.81773 



step: 772 loss: -28562  time/step: 0.54828  accept:   1     eps: 0.18806  avg_action: 39.483  avg_top_charge: 0.0015464  avg_plaq: 0.82374 



step: 773 loss: -26977  time/step: 0.51857  accept: 0.69942  eps: 0.18805  avg_action: 39.678  avg_top_charge: -0.00013414  avg_plaq: 0.82286 



step: 774 loss: -26834  time/step: 0.57384  accept:   1     eps: 0.18804  avg_action: 43.473  avg_top_charge: -0.0041534  avg_plaq: 0.805



step: 825 loss: -29941  time/step: 0.53834  accept:   1     eps: 0.1875  avg_action: 34.99   avg_top_charge: -0.033777  avg_plaq: 0.84379 



step: 826 loss: -24398  time/step: 0.50156  accept: 0.56492  eps: 0.18749  avg_action: 33.303  avg_top_charge: 0.047718  avg_plaq: 0.85133 



step: 827 loss: -24961  time/step: 0.6007  accept: 0.69229  eps: 0.18748  avg_action: 37.237  avg_top_charge: 0.021737  avg_plaq: 0.83376 



step: 828 loss: -26013  time/step: 0.49083  accept: 0.60499  eps: 0.18747  avg_action: 41.162  avg_top_charge: 0.032729  avg_plaq: 0.81624 



step: 829 loss: -28626  time/step: 0.61705  accept:   1     eps: 0.18745  avg_action: 31.683  avg_top_charge: -0.049447  avg_plaq: 0.85856 



step: 830 loss: -27602  time/step: 0.5569  accept:   1     eps: 0.18744  avg_action: 34.921  avg_top_charge: 0.051189  avg_plaq: 0.8441 



step: 831 loss: -29421  time/step: 0.48985  accept:   1     eps: 0.18743  avg_action: 37.282  avg_top_charge: -0.016005  avg_plaq: 0.83356 



st



step: 882 loss: -25012  time/step: 0.58758  accept:   1     eps: 0.18704  avg_action: 31.53   avg_top_charge: 0.024816  avg_plaq: 0.85924 



step: 883 loss: -27574  time/step: 0.54174  accept:   1     eps: 0.18704  avg_action: 33.125  avg_top_charge: 0.056124  avg_plaq: 0.85212 



step: 884 loss: -29480  time/step: 0.60334  accept:   1     eps: 0.18704  avg_action: 35.745  avg_top_charge: 0.11045  avg_plaq: 0.84043 



step: 885 loss: -27527  time/step: 0.58643  accept: 0.96941  eps: 0.18703  avg_action: 37.53   avg_top_charge: 0.079608  avg_plaq: 0.83245 



step: 886 loss: -26323  time/step: 0.55194  accept: 0.70684  eps: 0.18703  avg_action: 42.782  avg_top_charge: 0.18215  avg_plaq: 0.80901 



step: 887 loss: -27133  time/step: 0.57236  accept: 0.65164  eps: 0.18703  avg_action: 36.816  avg_top_charge: 0.09136  avg_plaq: 0.83564 



step: 888 loss: -30410  time/step: 0.59547  accept:   1     eps: 0.18703  avg_action: 35.413  avg_top_charge: 0.047359  avg_plaq: 0.84191 



step



step: 939 loss: -27014  time/step: 0.57332  accept:   1     eps: 0.18641  avg_action: 38.529  avg_top_charge: 0.063753  avg_plaq: 0.828  



step: 940 loss: -27490  time/step: 0.63038  accept: 0.59101  eps: 0.1864  avg_action: 37.039  avg_top_charge: 0.11662  avg_plaq: 0.83465 



step: 941 loss: -26055  time/step: 0.48221  accept: 0.53151  eps: 0.18639  avg_action: 35.53   avg_top_charge: 0.046771  avg_plaq: 0.84138 



step: 942 loss: -27279  time/step: 0.56098  accept: 0.70817  eps: 0.18638  avg_action: 42.036  avg_top_charge: 0.043994  avg_plaq: 0.81234 



step: 943 loss: -29365  time/step: 0.55908  accept:   1     eps: 0.18636  avg_action: 33.808  avg_top_charge: 0.082118  avg_plaq: 0.84907 



step: 944 loss: -28066  time/step: 0.4939  accept: 0.94759  eps: 0.18635  avg_action: 32.01   avg_top_charge: 0.018315  avg_plaq: 0.8571 



step: 945 loss: -29601  time/step: 0.60794  accept: 0.73124  eps: 0.18633  avg_action: 26.199  avg_top_charge: -0.018834  avg_plaq: 0.88304 



ste



step: 996 loss: -26456  time/step: 0.47778  accept: 0.65704  eps: 0.18552  avg_action: 40.322  avg_top_charge: 0.14778  avg_plaq: 0.81999 



step: 997 loss: -26673  time/step: 0.58097  accept: 0.68874  eps: 0.1855  avg_action: 35.926  avg_top_charge: 0.12555  avg_plaq: 0.83962 



step: 998 loss: -26383  time/step: 0.49104  accept: 0.66634  eps: 0.18549  avg_action: 30.681  avg_top_charge: 0.11987  avg_plaq: 0.86303 



step: 999 loss: -27185  time/step: 0.49298  accept: 0.85378  eps: 0.18548  avg_action: 31.491  avg_top_charge: -0.0082048  avg_plaq: 0.85941 

Training complete.
Saved checkpoint to: /Users/saforem2/ANL/l2hmc/gauge_logs_eager/run_19/ckpt-21
avg_plaquettes: [0.8698861 0.8489416]



In [58]:
avg_plaq_arr = np.array(model.average_plaquettes_arr)

In [59]:
avg_plaq_arr.shape

(398, 2)

In [62]:
avg_plaq_arr.mean(axis=0)

array([0.81270105, 0.8250402 ], dtype=float32)

In [63]:
u1_plaq_exact(params['beta'])

0.8411037970508364

In [43]:
%debug

> /Users/saforem2/ANL/l2hmc/l2hmc/l2hmc_eager/gauge_dynamics_eager.py(476)compute_loss()
    474                       * z_accept_prob + eps)
    475         else:
--> 476             raise ValueError("`metric` argument must be one of `l1`, `l2`, or"
    477                              " `cos`.")
    478 

ipdb> ll
    432 def compute_loss(dynamics, x, scale=0.1, eps=1e-4, metric='l2'):
    433     """Compute loss defined in Eq. (8) of paper."""
    434 
    435     z = tf.random_normal(tf.shape(x))  # Auxiliary variable
    436 
    437     #  if tf.executing_eagerly():
    438     #      print("Wrapping `dynamics.apply_transition` in `tfe.defun`...")
    439     #      apply_transition = tfe.defun(dynamics.apply_transition)
    440     #      _x, _, x_accept_prob, x_out = apply_transition(x)
    441     #      _z, _, z_accept_prob, _ = apply_transition(z)
    442     #  else:
    443     _x, _, x_accept_prob, x_out = dynamics.apply_transition(x)
    444     _z, _, z_accept_prob, _ =

# OLD

## Helper Functions

In [5]:
def train_one_iter(dynamics, x, optimizer, loss_fn=l2hmc.compute_loss, 
                   scale=0.1, eps=1e-4, metric='l2', clip_value=10, 
                   global_step=None):
    loss, grads, out, accept_prob = l2hmc.loss_and_grads(
        dynamics, x, loss_fn=loss_fn, scale=scale, eps=eps, metric=metric
    )
    gradients, _ = tf.clip_by_global_norm(grads, clip_value)
    optimizer.apply_gradients(
        zip(grads, dynamics.trainable_variables), global_step=global_step
    )
    return loss, out, accept_prob, gradients

In [6]:
def write_run_data(file_path, data, write_mode='a'):
    with open(file_path, write_mode) as f:
        f.write('\n')
        info_str = (f"step: {data['step']:<3g} loss: {data['loss']:^6.5g} "
                    f" time/step: {data['time']:^6.5g} "
                    f" accept: {data['accept_prob']:^6.5g} "
                    f" eps: {data['eps']:^6.5g} "
                    f" avg_S: {data['avg_S']:^6.5g} "
                    f" avg_topQ: {data['avg_top_charge']:^6.5g} "
                    f" avg_plaq: {data['avg_plaq']:^6.5g} \n")
        f.write(info_str)
        f.write('\n')
        f.write('avg_plaquettes: {}\n'.format(data['avg_plaquettes']))
        f.write('topological_charges: {}\n'.format(data['top_charges']))
        f.write('total_actions: {}\n'.format(data['total_actions']))
        f.write((len(info_str) + 1)*'-')

In [7]:
def write_run_parameters(file_path, parameters):
    with open(file_path, 'w') as f:
        f.write('Parameters:\n')
        f.write(80 * '-' + '\n')
        for key, val in parameters.items():
            f.write(f'{key}: {val}\n')
        f.write(80*'=')
        f.write('\n')

In [8]:
def print_run_data(data):
    print(f"\nstep: {data['step']:<3g} loss: {data['loss']:^6.5g} "
          f" time/step: {data['time']:^6.5g} "
          f" accept: {data['accept_prob']:^6.5g} "
          f" eps: {data['eps']:^6.5g} "
          f" avg_S: {data['avg_S']:^6.5g} "
          f" avg_topQ: {data['avg_top_charge']:^6.5g} "
          f" avg_plaq: {data['avg_plaq']:^6.5g} \n")
    print('avg_plaquettes: {}\n'.format(data['avg_plaquettes']))

In [9]:
def save_run_data(checkpointer, log_dir, files, data):
    saved_path = checkpointer.save(file_prefix=os.path.join(log_dir,
                                                            "ckpt"))
    print(f"Saved checkpoint to: {saved_path}")
    np.save(files['avg_plaquettes_file'], data['avg_plaquettes_arr'])
    np.save(files['total_actions_file'], data['total_actions_arr'])
    np.save(files['top_charges_file'], data['top_charges_arr'])
    np.save(files['samples_file'], data['samples'])

In [10]:
def write_summaries(summary_writer, data):
    with summary_writer.as_default():
        with tf.contrib.summary.always_record_summaries():
            tf.contrib.summary.scalar("Training_loss", data['loss'],
                                      step=data['global_step'])
            tf.contrib.summary.scalar("avg_plaquettes", data['avg_plaq'],
                                      step=data['global_step'])
            tf.contrib.summary.scalar("total_actions", data['avg_S'],
                                      step=data['global_step'])
            tf.contrib.summary.scalar("top_charges", data['avg_top_charge'],
                                      step=data['global_step'])

In [11]:
def exact_plaquette_average(beta):
    return i1(beta) / i0(beta)

## Construct GaugeLattice with $U(1)$ gauge group

$$ S[\theta] = \beta \sum_{x;\,\, \nu\neq\mu} 1 - \cos(\theta_{\mu\nu})$$
with $\theta_{\mu\nu} \equiv \theta_{\mu}(x) + \theta_{\nu}(x +\hat \mu) - \theta_{\mu}(x + \hat \nu) - \theta_{\nu}(x)$ 

### Run L2HMC for $U(1)$ gauge model

In [12]:
##########################  Parameters  #####################################
# n_steps: number of leapfrog steps, eps: initial step size for dynamics
# loss_scale: scaling factor (lambda^2 in paper) in loss objective
# loss_eps: for numeric stability in loss function
# beta: inverse coupling strength
##############################################################################
params = {
    'time_size': 8,
    'space_size': 8,
    'dim': 2,
    'beta': 3.5,
    'num_samples': 2,
    'n_steps': 10,
    'eps': 0.05,
    'loss_scale': 0.1,
    'loss_eps': 1e-4,
    'learning_rate': 1e-4,
    'learning_rate_decay_steps': 100,
    'learning_rate_decay_rate': 0.96,
    'train_iters': 500,
    'record_loss_every': 50,
    'save_steps': 50,
    'clip_value': 100,
    'rand': False,
    'conv_net': True,
    'metric': 'l2'
}

In [15]:
u1_lattice = GaugeLattice(time_size=params['time_size'], 
                          space_size=params['space_size'], 
                          dim=params['dim'], 
                          beta=params['beta'],
                          link_type='U1', 
                          num_samples=params['num_samples'], 
                          rand=params['rand'])
if params['conv_net']:
    u1_samples_tensor = tf.convert_to_tensor(u1_lattice.samples, 
                                             dtype=tf.float32)
else:
    flat_samples = [sample.flatten() for sample in u1_lattice.samples]
    u1_samples_tensor = tf.convert_to_tensor(np.stack(flat_samples), 
                                             dtype=tf.float32)

# Construct dynamics object
u1_energy_fn = u1_lattice.get_energy_function(u1_samples_tensor)
u1_dynamics = l2hmc.GaugeDynamicsEager(lattice=u1_lattice, 
                                       n_steps=params['n_steps'],
                                       eps=params['eps'],
                                       minus_loglikelihood_fn=u1_energy_fn, 
                                       conv_net=params['conv_net'], 
                                       hmc=False)
total_actions = []
average_plaquettes = []
topological_charges = []
samples = u1_samples_tensor

In [16]:
# Create new log_dir with new run number
root_dir = '../../U1_logs/'
if not os.path.exists(root_dir):
    os.makedirs(root_dir)

log_dirs = os.listdir(root_dir)
try:
    run_nums = [int(i.split('_')[-1]) for i in log_dirs if i.startswith('run')]
    run_num = max(run_nums) + 1
except:
    run_num = 1
    
log_dir = f'../../U1_logs/run_{run_num}'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
print(f'{log_dir}')
    
run_files = {
    'parameters_file': os.path.join(log_dir, 'parameters.txt'),
    'run_info_file': os.path.join(log_dir, 'run_info.txt'),
    'avg_plaquettes_file':  os.path.join(log_dir, 'average_plaquettes.npy'),
    'total_actions_file': os.path.join(log_dir, 'total_actions.npy'),
    'top_charges_file': os.path.join(log_dir, 'topological_charges.npy'),
    'samples_file': os.path.join(log_dir, 'samples.npy'),
}

../../U1_logs/run_226


In [18]:
global_step = tf.train.get_or_create_global_step()
_ = global_step.assign(1)

learning_rate = tf.train.exponential_decay(learning_rate=params['learning_rate'], 
                                           global_step=global_step, 
                                           decay_steps=params['learning_rate_decay_steps'],
                                           decay_rate=params['learning_rate_decay_rate'], 
                                           staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
checkpointer = tf.train.Checkpoint(
    optimizer=optimizer, dynamics=u1_dynamics, global_step=global_step
)
summary_writer = tf.contrib.summary.create_file_writer(log_dir)
loss_fn = l2hmc.compute_loss

print(u1_plaq_exact(params['beta']))

0.8411037970508364


In [19]:
################### SAVE INFO ABOUT INITIAL STATE OF LATTICE ###################
start_step = global_step.numpy()
observables = np.array(u1_lattice.calc_plaq_observables(samples))
_total_actions = observables[:, 0]
_avg_plaquettes = observables[:, 1]
_top_charges = observables[:, 2]

total_actions.append(_total_actions)
average_plaquettes.append(_avg_plaquettes)
topological_charges.append(_top_charges)

data = {
    'step': 0,
    'global_step': global_step.numpy(),
    'loss': 0.,
    'time': 0.,
    'accept_prob': 0.,
    'eps': u1_dynamics.eps.numpy(),
    'avg_S': np.mean(_total_actions),
    'avg_top_charge': np.mean(_top_charges),
    'avg_plaq': np.mean(_avg_plaquettes),
    'avg_plaquettes': _avg_plaquettes,
    'top_charges': _top_charges,
    'total_actions': _total_actions,
    'avg_plaquettes_arr': average_plaquettes,
    'top_charges_arr': topological_charges,
    'total_actions_arr': total_actions,
    'samples': samples.numpy()
}

_ = print_run_data(data)
_ = write_run_data(run_files['run_info_file'], data, 'w')
_ = write_run_parameters(run_files['parameters_file'], params)


step: 0   loss:   0     time/step:   0     accept:   0     eps:  0.05   avg_S:   0     avg_topQ:   0     avg_plaq:   1    

avg_plaquettes: [1. 1.]



In [ ]:
############################ RUN L2HMC ALGORITHM ############################
t0 = time.time()
for i in range(start_step, 1000):
    t1 = time.time()
    loss, samples, accept_prob, grads = train_one_iter(
        u1_dynamics,
        samples,
        optimizer,
        loss_fn=loss_fn,
        scale=loss_scale,
        metric=metric,
        eps=loss_eps,
        clip_value=clip_value,
        global_step=global_step
    )
    observables = np.array(u1_lattice.calc_plaq_observables(samples))
    _total_actions = observables[:, 0]
    _avg_plaquettes = observables[:, 1]
    _top_charges = observables[:, 2]
    
    total_actions.append(_total_actions)
    average_plaquettes.append(_avg_plaquettes)
    topological_charges.append(_top_charges)
    
    data = {
        'step': i,
        'global_step': global_step.numpy(),
        'loss': loss.numpy(),
        'time': time.time() - t1,
        'accept_prob': accept_prob.numpy().mean(),
        'eps': u1_dynamics.eps.numpy(),
        'avg_S': np.mean(_total_actions),
        'avg_top_charge': np.mean(_top_charges),
        'avg_plaq': np.mean(_avg_plaquettes),
        'avg_plaquettes': _avg_plaquettes,
        'top_charges': _top_charges,
        'total_actions': _total_actions,
        'avg_plaquettes_arr': np.array(average_plaquettes),
        'top_charges_arr': np.array(topological_charges),
        'total_actions_arr': np.array(total_actions),
        'samples': samples.numpy()
    }
    
    _ = write_run_data(run_files['run_info_file'], data, 'a')
    _ = print_run_data(data)
    
    if i % record_loss_every == 0:
        write_summaries(summary_writer, data)

    if i % save_steps == 0:
        save_run_data(checkpointer, log_dir, run_files, data)

In [819]:
avg_plaqs_arr = np.array(average_plaquettes)
_avg_plaqs_arr = np.mean(avg_plaqs_arr, axis=0)
avg_plaq, avg_plaq_err = calc_avg_vals_errors(avg_plaqs_arr[-400:], num_blocks=10)
print(f'avg_plaq (mean from arr): {np.mean(_avg_plaqs_arr)}')
print(f'avg_plaq: {avg_plaq} +/- {avg_plaq_err}')

avg_plaq (mean from arr): 0.7276734113693237
avg_plaq: 0.7387776374816895 +/- 0.0036369227224803927


In [279]:
calc_avg_vals_errors(_avg_plaqs_arr, num_blocks=50)

(0.7270305, 0.0037069468864631464)

### Gradients check

Want to test that the value of the gradient ($\partial_x U(x)$) is the same when  
calculated using either:
1. Tensorflow's automatic differentiation (`tf.gradients`), or
2. The analytic expression (Eq. 17) from [reference](https://arxiv.org/pdf/hep-lat/9809160.pdf):
$$\partial_x U(x) = \beta \sum_{\nu \neq \mu} \sin \theta_{\mu\nu}(x - \hat\nu) - \sin \theta_{\mu\nu}(x)$$

In [210]:
# fake data used to compare values of the gradient of the potential function
samples_ = tf.convert_to_tensor(
    [np.arange(128).reshape(u1_lattice.links.shape) for _ in range(4)],
    dtype=tf.float32
)

In [218]:
# using tensorflow's automatic differentiation 
tf_grad = u1_dynamics.grad_potential(samples) 
# 276 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

276 ms ± 15.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [219]:
# using the analytic expression
# (Eq. 17 from https://arxiv.org/pdf/hep-lat/9809160.pdf)
exact_grad = u1_lattice.grad_action(samples)
# 503 ms ± 33.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

503 ms ± 33.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [217]:
print('exact grad:')
print(exact_grad[0].reshape(u1_lattice.links.shape)[:4, :4, 0])
print('\n')
print('tf grad:')
print(tf_grad[0].numpy().reshape(u1_lattice.links.shape)[:4, :4, 0])

exact grad:
[[-0.97712278  3.26007509 -1.55236864 -3.12323356]
 [-3.12247324 -0.2006627  -1.83291686 -0.71278107]
 [ 0.98088837  0.01663715 -1.67887139  0.48193204]
 [ 4.18461895 -1.57589948 -2.16718554  0.24280304]]


tf grad:
[[-0.9771228   3.260075   -1.5523686  -3.1232338 ]
 [-3.1224732  -0.20066273 -1.8329167  -0.7127812 ]
 [ 0.98088837  0.01663721 -1.6788715   0.48193204]
 [ 4.184619   -1.5758996  -2.1671855   0.24280298]]


In [220]:
print('exact grad:')
print(exact_grad[0].reshape(u1_lattice.links.shape)[:4, :4, 1])
print('\n')
print('tf grad:')
print(tf_grad[0].numpy().reshape(u1_lattice.links.shape)[:4, :4, 1])

exact grad:
[[-0.19238535 -2.73329401  0.84687883  3.08645296]
 [ 0.24397749  3.70471263  3.98526478  1.57481074]
 [-2.52266145 -2.73996425 -2.89400959 -4.0887208 ]
 [-0.34991017  1.24262476  1.73093772  1.97006512]]


tf grad:
[[-0.19238311 -2.7332976   0.84688044  3.086453  ]
 [ 0.2439785   3.7047162   3.9852643   1.5748118 ]
 [-2.5226645  -2.7399645  -2.8940098  -4.088723  ]
 [-0.349913    1.2426238   1.7309377   1.9700668 ]]


In [25]:
################  tests  #############################
#u1_lattice._total_action()
#u1_lattice._average_plaquette()
#u1_lattice.total_action(u1_samples)
#u1_lattice.total_action()
#u1_lattice.average_plaquette(u1_samples)
#u1_lattice.average_plaquette()
#---------------------------------------------
#_momentum = tf.random_normal(tf.shape(u1_samples))
#_potential = np.array(u1_dynamics.potential(u1_samples_tensor))
#_kinetic = u1_dynamics.kinetic(_momentum)
#_grad_potential = u1_dynamics.grad_potential(u1_samples_tensor)
#print(_potential); print('\n')
#print(_kinetic); print('\n')
#print(_grad_potential)
#print(_kinetic.numpy()); print('\n')
#print(_hamiltonian.numpy()); print('\n')
#print(_grad_potential[0][:10])
#---------------------------------------------
#site = u1_lattice.get_random_site()
#u = np.random.randint(u1_lattice.dim)
#v = np.random.randint(u1_lattice.dim)
#plaq = u1_lattice.plaquette_operator(site, u, v)
#---------------------------------------------
#u1_lattice.total_action(u1_samples_tensor)
#u1_lattice.average_plaquette(u1_samples_tensor)
#transition_out = u1_dynamics.apply_transition(u1_samples_tensor)
#x_post, p_post, accept_prob, x_out = transition_out
#loss, x_out, x_accept_prob = l2hmc.compute_loss(u1_dynamics, u1_samples_tensor)
#x_accept_prob
###############################################

## Heatbath Algorithm

In [14]:
time_size, space_size, dim, beta, num_samples = (16, 16, 2, 4., 5)
u1_lattice = GaugeLattice(time_size, space_size, dim, beta,
                          link_type='U1', num_samples=num_samples)
u1_samples = [sample.flatten() for sample in u1_lattice.samples]
u1_samples_tensor = tf.constant(np.stack(u1_samples), dtype=tf.float32)

eq_steps = 5000
acceptances = []
action_arr = [u1_lattice._total_action()]
avg_plaq_arr = [u1_lattice._average_plaquette()]
for i in range(eq_steps):
    action = u1_lattice._total_action()
    avg_plaq = u1_lattice._average_plaquette()
    change = avg_plaq - avg_plaq_arr[-1]
    avg_plaq_arr.append(avg_plaq)
    action_arr.append(action)
    print(f"Step: {i:<5g}\t action: {action:<8.4g}\t "
          f"avg plaq: {avg_plaq:<8.4g}\t change: {change:<8.4g}")
    accept = 0
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            accept += u1_lattice._update_link(site, d)
    acceptances.append(accept)
# 12.2s for 500 equilibration steps

avg_plaq_arr = [0]
p = k - j
for k in range(j, 40000):
    avg_plaq = u1_lattice._average_plaquette()
    change = avg_plaq - avg_plaq_arr[p-1]
    avg_plaq_arr.append(avg_plaq)
    print(f"Step: {k:<5g}: avg plaq: {avg_plaq:>12.4g} change: {change:12.4g}")
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            _ = u1_lattice._update_link(site, d)

num_acceptances = 0
measure_steps = 10000
avg_plq = np.zeros(measure_steps)
for step in range(measure_steps):
    for site in u1_lattice.iter_sites():
        for d in range(u1_lattice.dim):
            num_acceptances += u1_lattice._update_link(site, d)
    avg_plq[step] = u1_lattice._average_plaquette()

## OLD

### GMM Model (test L2HMC)

In [121]:
sigmas, distribution = gen_ring(1., var=0.02, nb_mixtures=4)

gmm_potential = distribution.get_energy_function()
gmm_dynamics = _l2hmc.Dynamics(x_dim=2, minus_loglikelihood_fn=gmm_potential,
                               n_steps=25, eps=0.1)

samples = distribution.get_samples(200)

_position = tf.convert_to_tensor(samples, dtype=tf.float32)
_momentum = tf.random_normal(tf.shape(_position))
_hamiltonian = gmm_dynamics.hamiltonian(_position, _momentum)

grad_pot = gmm_dynamics.grad_potential(_position, _momentum)

grad_pot.shape

gmm_dynamics.masks[0]

scale, translation, transformed = gmm_dynamics.position_fn([_position, _momentum, gmm_dynamics._get_time(0)])

scale.shape

_position.shape

gmm_dynamics.masks[0].shape

gmm_dynamics.masks[0] * _position

### Construct GaugeLattice with SU(3) gauge group

In [ ]:
time_size = 2
space_size = 4
dim = 4
beta = 1.
link_type = 'SU3' 
batch_size = 3
gauge_lattice = GaugeLattice(time_size, space_size, dim, beta, link_type)
# create `num_samples` random samples of GaugeLattice.links
links_samples = gauge_lattice.get_links_samples(batch_size, link_type=link_type)

In [ ]:
gauge_energy_fn = gauge_lattice.get_energy_function()
gauge_dynamics = l2hmc.GaugeDynamics(gauge_lattice, 
                                     minus_loglikelihood_fn=gauge_energy_fn, 
                                     batch_size=3, n_steps=5, eps=0.1)

In [ ]:
gauge_lattice.links.shape

In [ ]:
potential_arr = gauge_dynamics.potential(links_samples, batch_size)

[i.numpy() for i in potential_arr]

In [ ]:
_momentum = tf.random_normal(tf.shape(links_samples))

In [ ]:
gauge_dynamics.kinetic(_momentum).numpy()

In [ ]:
_x = links_samples
#_momentum = tf.random_normal(tf.shape(_x))
_hamiltonian = gauge_dynamics.hamiltonian(_x, _momentum)
_hamiltonian

### Construct IsingLattice

In [ ]:
ising_batch_size = 10
ising_lattice = IsingLattice(3, 4)
ising_samples = [ising_lattice._randomize() for _ in range(ising_batch_size)]

In [ ]:
ising_energy_fn = ising_lattice.get_energy_function()
ising_dynamics = l2hmc.LatticeDynamics(ising_lattice, 
                                       minus_loglikelihood_fn=ising_energy_fn,
                                       batch_size=ising_batch_size, 
                                       n_steps=10, eps=0.1)
#dynamics = l2hmc.LDynamics(latt.sites.shape, minus_loglikelihood_fn=energy_fn, n_steps=10, eps=0.1)

In [ ]:
ising_dynamics.potential(samples, batch_size)

In [ ]:
_iposition = ising_samples
_imomentum = tf.random_normal(tf.shape(_iposition))
_ihamiltonian = dynamics.hamiltonian(_iposition, _imomentum)
_ihamiltonian

In [ ]:
_isample = _iposition[0].reshape(ising_lattice.num_sites)
#dynamics.grad_potential(np.array(_position).reshape(-1, lattice.num_sites))

In [ ]:
grad_pot = dynamics.grad_potential(ising_samples)

In [ ]:
ising_jacobian = jacobian(dynamics.potential, ising_samples)

In [ ]:
grad_fn = tfe.gradients_function(lattice._calc_energy, params=[0])

In [ ]:
_jacobian = jacobian(dynamics.potential, _position)

In [ ]:
lattice.calc_energy(_position, batch_size)

In [ ]:
#dynamics.position_fn(momentum, latt.sites.flatten()[:], dynamics)
#dynamics._forward_lf(latt.sites.flatten()[:], momentum, 0)
dynamics._forward_lf(np.array(_position).reshape(-1, lattice.num_sites),
                     np.array(_momentum).reshape(-1, lattice.num_sites), 1)